In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -q transformers 
!pip install -q cdifflib 
!pip install -q pandas
!pip install -q IPython
!pip install -q datasets
!pip install -q mecab-python3
!pip install -q unidic-lite
!pip install -q pykakasi==2.2.1
!pip install tqdm
!python -m -q unidic download
!pip install --upgrade IPython
!pip install jiwer
exit()

In [ ]:
import pandas as pd
import librosa
from jiwer import wer as WordErrorRate
from jiwer import cer as CharErrorRate
import torch
from transformers import Wav2Vec2Tokenizer, Wav2Vec2ForCTC, Wav2Vec2Processor
import torchaudio
import MeCab
import re
import pykakasi
from tqdm.notebook import tqdm_notebook
from datasets import load_dataset, Dataset, load_metric
import IPython.display as display
import requests
import time
from datetime import timedelta
import os

In [ ]:
model_path = '/content/drive/MyDrive/livetranslator/models/wav2vec2-model-jp-2'
pkl_parent_folder = '/content/drive/MyDrive/livetranslator/pickle'

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
#pykakasi has deprecation warning and i don't know how to fix 

In [ ]:
#clean the data!!!!!
import pykakasi #package to convert japanese to hiragana
import MeCab
import torchaudio
import json
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“\‘\”\�‘、。．！，_・―─~｢｣『』\\\\※\[\]\{\}「」〇？…]' #special characters that isn't in japanese
wakati = MeCab.Tagger("-Owakati")
kakasi = pykakasi.kakasi()
#hiragana
kakasi.setMode('J', 'H') #kanji
kakasi.setMode('K', 'H') #katakana
conv = kakasi.getConverter()
FULLWIDTH_TO_HALFWIDTH = str.maketrans(
  '　０１２３４５６７８９ａｂｃｄｅｆｇｈｉｊｋｌｍｎｏｐｑｒｓｔｕｖｗｘｙｚＡＢＣＤＥＦＧＨＩＪＫＬＭＮＯＰＱＲＳＴＵＶＷＸＹＺ！゛＃＄％＆（）＊＋、ー。／：；〈＝〉？＠［］＾＿‘｛｜｝～',
  ' 0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&()*+,-./:;<=>?@[]^_`{|}~',
)
emoticon_pattern = re.compile("["
        u"\U00012600-\U000126FF"  #miscellaneous symbol
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)

symbol_pattern = re.compile('['
        r'\u2190-\u21FF' #arrow
        r'\u27F0-\u27FF' #arrow 2
        r'\u2900-\u297F' #arrow 3
        r'\u1F800-\u1F8FF' #arrow 4
        r'\u1F800-\u1F8FF' #arrow 5
        r'\u2000-\u206F' #punctuation
        r'\u20A0-\u20CF' #currency
        r'\u0391-\u03C9' #greek
        r'\u2200-\u22FF' #math operator
        r'\u2600-\u26FF' #misc
        r'\u2500-\u257F' #box 
        r'\u25A0-\u25FF' #shapes
        r'\u301C' #idk what's dis
        ']+', re.UNICODE)

def get_index_occurrences(list_of_elems, elem):
  index_pos_list = []
  for i in range(len(list_of_elems)):
      if list_of_elems[i] == elem:
          index_pos_list.append(i)
  return index_pos_list

def preprocessData(batch):
  batch['sentence'] = batch['sentence'].translate(FULLWIDTH_TO_HALFWIDTH)
  batch["sentence"] = re.sub(emoticon_pattern,' ', batch["sentence"])
  batch["sentence"] = re.sub(symbol_pattern,' ', batch["sentence"])
  batch["sentence"] = re.sub(chars_to_ignore_regex,' ', batch["sentence"]).lower()  #remove special char
  batch["sentence"] = wakati.parse(batch["sentence"]) #space
  batch["sentence"] = conv.do(batch["sentence"]) #convert to hiragana
  batch["sentence"] = " ".join(batch["sentence"].split())+" " #remove space

  speech_array, sampling_rate = torchaudio.load(batch["path"])
  batch["speech"] = torchaudio.functional.resample(speech_array, sampling_rate, 16000)[0].numpy()  
  return batch

def evaluate(batch):
  inputs = processor(batch["speech"], sampling_rate=16_000, return_tensors="pt", padding=True)
  if batch['sentence'] == '':
    batch['sentence'] = 'EMPTY'
  with torch.no_grad():
      logits = model(inputs.input_values.to("cuda"), attention_mask=inputs.attention_mask.to("cuda")).logits
  pred_ids = torch.argmax(logits, dim=-1)
  if processor.batch_decode(pred_ids) != '':
    batch["pred_strings"] = processor.batch_decode(pred_ids)
  else : batch["pred_strings"] = 'EMPTY'
  return batch

def pkl_concat(pkl_path_list: list):
  temp_df_list = []
  for pkl_path in pkl_path_list:
    temp_df = pd.read_pickle(pkl_path)
    temp_df_list.append(temp_df)
  return pd.concat(temp_df_list)

In [ ]:
#  eval_pkl_list = []
#  temp_eval_pkl_list =[pkl_list for pkl_list in os.listdir(pkl_parent_folder) if pkl_list.startswith('eval')]
#  for temp_path in temp_eval_pkl_list : eval_pkl_list.append(os.path.join(pkl_parent_folder, temp_path))
eval_df = pd.read_pickle('/content/drive/MyDrive/livetranslator/pickle/datasetname.pkl')

model = Wav2Vec2ForCTC.from_pretrained('thunninoi/wav2vec2-japanese-vtuber').cuda()
processor = Wav2Vec2Processor.from_pretrained('thunninoi/wav2vec2-japanese-vtuber')
#dataset = Dataset.from_pandas(eval_df).remove_columns(['__index_level_0__'])
wer = load_metric('wer')
cer = load_metric('cer')

#dataset = dataset.map(preprocessData)
#dataset = Dataset.from_pandas(eval_df)
dataset = load_dataset('common_voice', 'ja', split = 'test')
dataset = dataset.map(preprocessData)
result = dataset.map(evaluate, batched=True, batch_size=8)
df = pd.DataFrame(result)
to_remove = get_index_occurrences(result['pred_strings'], '')
df.drop(to_remove, inplace = True)
pred_str = df.loc[:,'pred_strings'].tolist()
groundtruth = df.loc[:,'sentence'].tolist()
print("WER: {:2f}".format(100 * WordErrorRate(pred_str, groundtruth)))
print("CER: {:2f}".format(100 * CharErrorRate(pred_str, groundtruth)))

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Reusing dataset common_voice (/root/.cache/huggingface/datasets/common_voice/ja/6.1.0/a1dc74461f6c839bfe1e8cf1262fd4cf24297e3fbd4087a711bd090779023a5e)
Parameter 'function'=<function preprocessData at 0x7f3f5b7e9830> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/632 [00:00<?, ?ex/s]

  0%|          | 0/79 [00:00<?, ?ba/s]

WER: 33.659159
CER: 15.570237
